In [1]:
# 1) Load Dataset
# 2) Make dataset iterable
# 3)Create Model class
# 4) Instantiate Model Class
# 5) Instantiate Loss Class
# 6) Instantiate optimizer Class
# 7) Tran Model

In [2]:
# Step 1:-
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import  torchvision.datasets as dsets
from torch.autograd import Variable
# Variables is for instantiating and assigning variables to a object

In [3]:
# Reaad data

In [4]:
train_dataset=dsets.MNIST(root='./data' ,train=True,transform=transforms.ToTensor())#Train data
test_dataset=dsets.MNIST(root='./data' ,train=False,transform=transforms.ToTensor())#Test data

In [12]:
train_dataset.train_labels

tensor([ 5,  0,  4,  ...,  5,  6,  8])

In [5]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


In [6]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


In [7]:
# Make dataset iterable

In [13]:
batch_size=100

n_iters=3000

num_epochs=n_iters/(len(train_dataset)/batch_size)

train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [32]:
# Create Model Class

In [14]:
# All goodness from pytorch -nn.Module
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel,self).__init__() #Super inheritance function
#         Convolution-1
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5, stride=1,padding=2) 
#     #2dimension, in_channels=1 bcoz of mnist grayscale,    Output channels = 16(no of kernels)hence 16 feature maps
#  
        self.relu1=nn.ReLU()
#     After eveery convolution layer pass it through the non linearity, Relu is best non linear function


#         Max pool 1
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
#         Convolution 2
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
#     in_channels=16 and output-channels=32
        self.relu2=nn.ReLU()
#         MAxpool 2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
#         Fully connected 1 --- 32*7*7 has been calculated and 10 is the mnist dataset output
        self.fc1=nn.Linear(32*7*7,10)
        
    def forward(self,x):
#         Convolution 1
        out=self.cnn1(x)
        out=self.relu1(out)
        
#         MAxpool1
        out=self.maxpool1(out)
        
#         Convolution 2
        out=self.cnn2(out)
        out=self.relu2(out)
#         Maxpool 2
        out=self.maxpool2(out)
#         Resize
# Original size (100,32,7,7) Input size 
# out.size(0):-100 is the batch size of 100
# New out size (100,32*7*7) , this is what linear function requires, we cannot feed a linear funcion of 3d here but 
# only 1 single dimension here.
        out=out.view(out.size(0),-1)
#     -1 means reshape to the remaining value which is 32*7*7 here, so that output can be feed to the linear function
        
        out=self.fc1(out)
        
        return out
        
        

# Step 4 :-Instantiate Model Class

In [15]:
model=CNNModel()

In [16]:
# Step 5:-Instantiate Loss class
# CNN-CrossEntropyLoss
# Feedforward NN- CrossEntropyLoss
# LogisticRegression- CrossEntropyLoss
# Linear Regression - MSE

In [17]:
criterion=nn.CrossEntropyLoss()

In [18]:
# Step 6 -Instantiate Optimizer Class


In [19]:
learning_rate=.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [20]:
# Parameters in depth

In [21]:
print(model)

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)


In [22]:
print(model.parameters())

<generator object Module.parameters at 0x0000003C9EEE8E60>


In [23]:
print(len(list(model.parameters())))

6


In [24]:
print(list(model.parameters())[0].size())
# 16 kernels of 5*5

torch.Size([16, 1, 5, 5])


In [25]:
print(list(model.parameters())[1].size())

torch.Size([16])


In [26]:
print(list(model.parameters())[2].size())
# 32 kernels of 5*5 and depth 16

torch.Size([32, 16, 5, 5])


In [27]:
print(list(model.parameters())[3].size())

torch.Size([32])


In [28]:
print(list(model.parameters())[4].size())
# Reshaping gives 1568

torch.Size([10, 1568])


In [29]:
print(list(model.parameters())[5].size())

torch.Size([10])


In [30]:
## Step 7 -Train Model
###### 1) convert input/labels into Variables 
######       cnn input (1,28,28)
######       Feedforward nn input(1,28*28)
###### 2) Clear Gradient buffers
###### 3) Get outputs given inputs
###### 4) Get Loss
###### 5)Get gradients wrt parameters
###### 6) Update parameters using gradients
###### 7)Repeat

In [31]:
iter=0
for epoch in range(int(num_epochs)):
    for i,(images,labels) in enumerate(train_loader):# 100 images per batch size, so 100 images at once . Every iteration
#         feed 100 images.
        images=Variable(images)
#         print(images.size())
        labels=Variable(labels)
        
        optimizer.zero_grad()
        
        outputs=model.forward(images)
        
        loss=criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter+=1
        
        if iter%500==0:
            correct=0
            total=0
            
            for images,labels in test_loader:
                images=Variable(images)
                outputs=model(images)
                
                _,predicted=torch.max(outputs.data,1)
                
                total+=labels.size(0)
                
                correct+=(predicted==labels).sum()
            accuracy=100*int(correct)/int(total)
            
            print('Iterations {}. Loss {}. Acuracy {}.'.format(iter,loss.data[0],accuracy))

C:\Users\ashis.panda\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iterations 500. Loss 0.4349491000175476. Acuracy 88.01.
Iterations 1000. Loss 0.3766486644744873. Acuracy 92.05.
Iterations 1500. Loss 0.3643883764743805. Acuracy 94.17.
Iterations 2000. Loss 0.0985739678144455. Acuracy 95.45.
Iterations 2500. Loss 0.16918666660785675. Acuracy 96.34.
Iterations 3000. Loss 0.05423809587955475. Acuracy 96.92.
